In [41]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


#https://colab.research.google.com/drive/1N3LvAO0AXV4kBPbTMX866OwJM9YS6Ji2?usp=sharing#scrollTo=fl5W1gg5Jhzz

df_players = pd.read_csv("./../data_scrapped/atp_players.csv")
df_matchs = pd.read_csv("./../data_formatted/training_dgl_dataset.csv")

In [22]:
df_players_indices = []
for index,row in df_players.iterrows():
    winmatchs = df_matchs[df_matchs.player1_id == row.player_id]
    lostmatchs = df_matchs[df_matchs.player2_id == row.player_id]
    if((len(winmatchs) + len(lostmatchs))>6):
        df_players_indices.append(index)


In [23]:
df_players_filtered = df_players.loc[df_players_indices]

In [30]:
df_players

,player_id,first_name,last_name,country_code,birthdate,birth_year,birth_month,birth_day,turned_pro,weight_kg,weight_lbs,height_cm,height_in,birthplace
0,ge73,Jacob,Grills,AUS,1995.06.18,1995.0,6.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN
1,rd34,Lukas,Ruepke,GER,1993.07.06,1993.0,7.0,6.0,NaN,85.0,187.0,180.0,71.0,"Bremen, Germany"
2,h917,Tigre,Hank,MEX,1991.09.30,1991.0,9.0,30.0,NaN,85.0,187.0,188.0,74.0,NaN
3,l836,Nick,Lindahl,AUS,1988.07.31,1988.0,7.0,31.0,2006.0,77.0,170.0,183.0,72.0,"Malmo, Sweden"
4,r388,Sergio,Roitman,ARG,1979.05.16,1979.0,5.0,16.0,1996.0,79.0,175.0,185.0,73.0,"Buenos Aires, Argentina"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,f0ff,Philip,Florig,GER,2003.09.11,2003.0,9.0,11.0,NaN,78.0,171.0,180.0,71.0,Aschaffenburg
3442,me56,Andres,Molteni,ARG,1988.03.15,1988.0,3.0,15.0,2005.0,70.0,154.0,180.0,71.0,"Buenos Aires, Argentina"
3443,d551,Farrukh,Dustov,UZB,1986.05.22,1986.0,5.0,22.0,2003.0,88.0,194.0,193.0,76.0,"Tashkent, Uzbekistan"
3444,s893,Martin,Slanar,AUT,1981.05.01,1981.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_players_filtered

,player_id,first_name,last_name,country_code,birthdate,birth_year,birth_month,birth_day,turned_pro,weight_kg,weight_lbs,height_cm,height_in,birthplace
3,l836,Nick,Lindahl,AUS,1988.07.31,1988.0,7.0,31.0,2006.0,77.0,170.0,183.0,72.0,"Malmo, Sweden"
4,r388,Sergio,Roitman,ARG,1979.05.16,1979.0,5.0,16.0,1996.0,79.0,175.0,185.0,73.0,"Buenos Aires, Argentina"
8,p280,Andrei,Pavel,ROU,1974.01.27,1974.0,1.0,27.0,1995.0,90.0,198.0,183.0,72.0,"Constanta, Romania"
9,p305,Albert,Portas,ESP,1973.11.15,1973.0,11.0,15.0,1994.0,78.0,172.0,188.0,74.0,"Barcelona, Spain"
12,c723,Flavio,Cipolla,ITA,1983.10.20,1983.0,10.0,20.0,2003.0,74.0,163.0,173.0,68.0,"Rome, Italy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,m0ac,Rudolf,Molleker,GER,2000.10.26,2000.0,10.0,26.0,2017.0,82.0,180.0,188.0,74.0,"Severodonetsk, Ukraine"
3415,m595,Max,Mirnyi,NaN,1977.07.06,1977.0,7.0,6.0,1994.0,93.0,205.0,196.0,77.0,"Minsk, Belarus"
3416,c582,Marco,Chiudinelli,SUI,1981.09.10,1981.0,9.0,10.0,2000.0,85.0,187.0,185.0,73.0,"Basel, Switzerland"
3428,n0bg,Luca,Nardi,ITA,2003.08.06,2003.0,8.0,6.0,NaN,80.0,176.0,185.0,73.0,Pesaro


In [31]:
#On construit le graph

# lecture des données attendues
# tensor1[1] -> tensor2[1] 
# le player index en tensor1 a joué contre le player au meme index en tensor 2
# y stocke le résultat du match


tensor1=[]
tensor2=[]

for index,row in df_players_filtered.iterrows():
    winmatchs = df_matchs[df_matchs.player1_id == row.player_id]
    #print(len(winmatchs))
    if len(winmatchs) > 0:
        #print(row.player_id)
        for index2, row2 in winmatchs.iterrows():
                if len(df_players_filtered.loc[df_players_filtered.player_id == row2.player2_id]) > 0:
                    tensor1.append(index)
                    tensor2.append(df_players_filtered.loc[df_players_filtered.player_id == row2.player2_id].index[0])

In [36]:
print(len(df_players_filtered))

749


In [37]:
import numpy
import random

labels = numpy.ones(len(tensor1))

indexes = random.sample(range(0, len(tensor1)-1), ((len(tensor1)-1)//2))

for index in indexes:
    temp = tensor1[index]
    tensor1[index] = tensor2[index]
    tensor2[index] = temp
    labels[index] = 0

In [38]:
df_players.shape

(3446, 14)

In [42]:
import torch_geometric.transforms as T
from torch_geometric.data import Data

# Create the heterogeneous graph data object:
#data = Data()

# Add the user nodes:
x = torch.tensor(list(df_players_filtered[["birth_year","weight_kg","height_cm"]].values),dtype=torch.float)  # [num_users, num_features_users]
x = torch.nan_to_num(x, nan=0.0)
#x = torch.ones(df_players.shape[0])
#x = torch.masked_select(x, ~torch.isnan(x))
#x = torch.ones(df_players.shape[0])

edge_index = torch.stack([torch.tensor(tensor1), torch.tensor(tensor2)], dim=0)
edge_attr = torch.Tensor(list(df_matchs[["player1_atprank","player1_oddsB365","player2_atprank","player2_oddsB365"]].values))
#y = torch.Tensor(list(df_matchs[["winner_player1"]].values))

labels = torch.tensor(labels, dtype=torch.long)

data = Data(x=x, edge_index=edge_index,edge_label=labels) 
#data = T.ToUndirected()(data)
# Add the movie nodes:
#data['movie'].x = movie_features  # [num_movies, num_features_movies]

data

C:\Users\chbou\AppData\Local\Temp\ipykernel_16800\3456287830.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  x = torch.tensor(list(df_players_filtered[["birth_year","weight_kg","height_cm"]].values),dtype=torch.float)  # [num_users, num_features_users]


Data(x=[749, 3], edge_index=[2, 50586], edge_label=[50586])

In [6]:
labels

tensor([0, 1, 1,  ..., 1, 0, 1])

In [43]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=False,
    neg_sampling_ratio=0
)(data)
train_data, val_data, test_data

(Data(x=[749, 3], edge_index=[2, 42999], edge_label=[42999], edge_label_index=[2, 42999]),
 Data(x=[749, 3], edge_index=[2, 42999], edge_label=[2529], edge_label_index=[2, 2529]),
 Data(x=[749, 3], edge_index=[2, 45528], edge_label=[5058], edge_label_index=[2, 5058]))

In [19]:
from torch_geometric.nn import GATConv,MLP
from torch_geometric.loader import DataLoader
from tqdm import tqdm

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(3, 128, edge_dim = 4)
        self.conv2 = GATConv(128, 64, edge_dim = 4)
        self.mlp = MLP(in_channels=128, hidden_channels=128, out_channels=1, num_layers=3)

    def forward(self, x, edge_index, edge_label_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr).relu()
        return self.mlp(torch.cat((x[edge_label_index[0]], x[edge_label_index[1]]), dim=-1))
    
model = Net().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_label_index, data.edge_attr)
    loss = criterion(out.squeeze(), data.edge_label.float())
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(data):
    model.eval()
    out = model(data.x, data.edge_index, data.edge_label_index, data.edge_attr)
    return roc_auc_score(data.edge_label.float(), out.squeeze())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train(train_data)
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, 'f'Test: {test_auc:.4f}')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3446 and 1x128)

In [18]:
# Specify the edge you want to predict (e.g., edge from node 0 to node 1)
node_0 = 3
node_1 = 664

print(df_players.iloc[node_0])
#print(df_players.iloc[node_0]["first_name","last_name"])

# Predict the direction of the edge
with torch.no_grad():
    model.eval()
    edge_label_index =  torch.tensor([[node_0, node_1], [node_1, node_0]], dtype=torch.long).t()
    print(edge_label_index)
    prediction = model(data.x, data.edge_index, edge_label_index, data.edge_attr).view(-1).sigmoid()
    print(prediction)

player_id                l836
first_name               Nick
last_name             Lindahl
country_code              AUS
birthdate          1988.07.31
birth_year             1988.0
birth_month               7.0
birth_day                31.0
turned_pro             2006.0
weight_kg                77.0
weight_lbs              170.0
height_cm               183.0
height_in                72.0
birthplace      Malmo, Sweden
Name: 3, dtype: object
tensor([[  3, 664],
        [664,   3]])
tensor([0.3646, 0.6394])
